In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-24 16:16:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.77MB/s    in 0.2s    

2022-04-24 16:16:12 (5.77 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles

# Read in the Review dataset as a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [6]:
#1. Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count is equal to or greater than 20 
total_votes_df = df.filter("total_votes>20")
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [7]:
#2. Filter the new DataFrame in Step 1 and create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.filter("helpful_votes/total_votes>=0.50")
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [8]:
#3. Filter the DataFrame in Step 2, and create a new DataFrame to retrieve all the rows where a review was written as part of the Vine program (paid)
paid_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
paid_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35119071|R2BQOD1R0228FN|B00H2RSA88|     405483618|Sleep Innovations...|       Furniture|          3|           17|         26|   Y|                N|An okay product. ...|Three-stars is co...| 2015-08-27|
|         US|   44737123| RC31RUPFOHBHQ|B0125QZ50G|     350975212|Zinus Viscolatex ...|       Furniture|          4|    

In [9]:
#4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_review_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
unpaid_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

**Helpful Reviews Analysis**

In [10]:
#5a. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
total_reviews = df.count()
total_helpful_reviews = helpful_votes_df.count()

#only using helpful reviews for analysis

#5-star reviews
five_star_reviews = helpful_votes_df.filter("star_rating==5").count()
five_star_percent = five_star_reviews/total_helpful_reviews*100

#Paid reviews
paid_reviews = paid_review_df.count()
five_star_paid_reviews = paid_review_df.filter("star_rating==5").count()
five_star_paid_percent = five_star_paid_reviews/paid_reviews*100

#Unpaid reviews
unpaid_reviews = unpaid_review_df.count()
five_star_unpaid_reviews = unpaid_review_df.filter("star_rating==5").count()
five_star_unpaid_percent = five_star_unpaid_reviews/unpaid_reviews*100

In [24]:
#Creating a summary DataFrame of helpful voted reviews
helpful_summary_df = spark.createDataFrame([
                                   ("Vine (paid)", paid_reviews, five_star_paid_reviews, five_star_paid_percent),
                                   ("Non-Vine (unpaid)", unpaid_reviews, five_star_unpaid_reviews, five_star_unpaid_percent),
], ["Review Type", "Total Reviews", "5 Star Reviews", "% 5 Star Reviews"])

helpful_summary_df.show()

+-----------------+-------------+--------------+-----------------+
|      Review Type|Total Reviews|5 Star Reviews| % 5 Star Reviews|
+-----------------+-------------+--------------+-----------------+
|      Vine (paid)|          130|            70|53.84615384615385|
|Non-Vine (unpaid)|        16820|          7956|47.30083234244947|
+-----------------+-------------+--------------+-----------------+



**All Reviews (helpful & non-helpful) Analysis**

In [26]:
#using all reviews for analysis
all_reviews = df.count()
#5-star reviews
five_star_df = df.filter("star_rating==5")
all_five_star_reviews = five_star_df.count()
all_five_star_percent = all_five_star_reviews/total_reviews*100

#Paid reviews
all_paid_df = df.filter(df["vine"] == "Y")
all_paid_reviews = all_paid_df.count()
all_paid_percent = all_paid_reviews/total_reviews*100

all_five_star_paid_reviews = all_paid_df.filter("star_rating==5").count()
all_five_star_paid_percent = all_five_star_paid_reviews/all_paid_reviews*100

#Unpaid reviews
all_unpaid_df = df.filter(df["vine"] == "N")
all_unpaid_reviews = all_unpaid_df.count()
all_unpaid_percent = all_unpaid_reviews/total_reviews*100

all_five_star_unpaid_reviews = all_unpaid_df.filter("star_rating==5").count()
all_five_star_unpaid_percent = all_five_star_unpaid_reviews/all_unpaid_reviews*100

In [25]:
#Creating a summary DataFrame of all reviews
all_summary_df = spark.createDataFrame([
                                   ("Vine (paid)", all_paid_reviews, all_five_star_paid_reviews, all_five_star_paid_percent),
                                   ("Non-Vine (unpaid)", all_unpaid_reviews, all_five_star_unpaid_reviews, all_five_star_unpaid_percent),
], ["Review Type", "Total Reviews", "5 Star Reviews", "% 5 Star Reviews"])

all_summary_df.show()

+-----------------+-------------+--------------+------------------+
|      Review Type|Total Reviews|5 Star Reviews|  % 5 Star Reviews|
+-----------------+-------------+--------------+------------------+
|      Vine (paid)|         2775|          1356| 48.86486486486486|
|Non-Vine (unpaid)|       789338|        446360|56.548652161684856|
+-----------------+-------------+--------------+------------------+



In [15]:
#Creating a summary DataFrame of 5 star reviews

below5_reviews = df.filter("star_rating!=5").count()
below5_percent = below5_reviews/total_reviews*100

stars_df = spark.createDataFrame([
                                   ("5-Star", all_five_star_reviews, all_five_star_percent),
                                   ("1-4 Stars", below5_reviews, below5_percent),
], ["Review Rating", "Count", "Percentage"])

stars_df.show()

+-------------+------+-----------------+
|Review Rating| Count|       Percentage|
+-------------+------+-----------------+
|       5-Star|447716|56.52173364153852|
|    1-4 Stars|344397|43.47826635846148|
+-------------+------+-----------------+

